In [2]:
import cv2 
import numpy as np
import pytesseract
pytesseract.pytesseract.tesseract_cmd = 'C:\\Program Files\\Tesseract-OCR\\tesseract.exe'  # need to be changed to the server path

#img = cv2.imread("D:\Reham\DATA\Graduation_project\OCR\data\wrap.jpg") # need to be replaced get_img function

#TODO: solve empty rectangular contour problem

def get_img_from_path(image_path):

    # ask how to take the image
    img = cv2.imread(image_path)
    if img is None:
        raise Exception("Image couldn't be read,try again")
    return img

In [3]:
def biggest_contour(contours):
    """get the rectangular contour of the ID picture

    Args:
        contours (list): a list of 10 biggest contours in ID pictures

    Returns:
        numpy.ndarray: a array of 4 points [[[x1,y1],[x2,y2],[x3,y3],[x4,y4]]] 
        that make the rectangular outer shape of the ID
    """
    biggest = np.array([])
    max_area = 0
    for i in contours:
        area = cv2.contourArea(i)
        if area > 10:
            peri = cv2.arcLength(i, True)
            approx = cv2.approxPolyDP(i, 0.015 * peri, True)
            print(len(approx),"approx")
            if area > max_area and len(approx) >= 4:
                biggest = approx
                max_area = area
                print(max_area,"max_area")
    print(len(biggest[:4]),"len biggest")

    return biggest[:4]

In [4]:
def get_contours(img):
    """change img to an ID image 600*400

    Args:
        img (MatLike): the image
    Raises:
        ValueError: when contours list are empty
        ValueError: when biggest rectangular contours are empty
        Exception:  if Wrapping failed

    Returns:
        MatLike: the ID image after operations(contours,warp perspective,resizing to 600*400)
    """
    img_original = img.copy()

    # Image modification
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray = cv2.bilateralFilter(gray, 20, 30, 30)
    edged = cv2.Canny(gray, 10, 20)

    # Contour detection
    contours, _ = cv2.findContours(edged.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    contours = sorted(contours, key=cv2.contourArea, reverse=True)[:10]
    
    if contours is None:
        raise ValueError("Contours are empty")

    #get the rectangular contour
    biggest = biggest_contour(contours)
    if biggest.size == 0:
        raise ValueError("Empty rectangular contours")
    #cv2.drawContours(img, [biggest], -1, (0, 255, 0), 3)

    # Pixel values in the original image
    points = biggest.reshape(4, 2)
    input_points = np.zeros((4, 2), dtype="float32")

    points_sum = points.sum(axis=1)
    input_points[0] = points[np.argmin(points_sum)]
    input_points[3] = points[np.argmax(points_sum)]

    points_diff = np.diff(points, axis=1)
    input_points[1] = points[np.argmin(points_diff)]
    input_points[2] = points[np.argmax(points_diff)]

    (top_left, top_right, bottom_right, bottom_left) = input_points
    bottom_width = np.sqrt(((bottom_right[0] - bottom_left[0]) ** 2) + ((bottom_right[1] - bottom_left[1]) ** 2))
    top_width = np.sqrt(((top_right[0] - top_left[0]) ** 2) + ((top_right[1] - top_left[1]) ** 2))
    right_height = np.sqrt(((top_right[0] - bottom_right[0]) ** 2) + ((top_right[1] - bottom_right[1]) ** 2))
    left_height = np.sqrt(((top_left[0] - bottom_left[0]) ** 2) + ((top_left[1] - bottom_left[1]) ** 2))

    # Output image size
    max_width = max(int(bottom_width), int(top_width))
    max_height = max(int(right_height), int(left_height))
    #max_height = int(max_width * 1.414)  # for A4

    # Desired points values in the output image
    converted_points = np.float32([[0, 0], [max_width, 0], [0, max_height], [max_width, max_height]])

    # Perspective transformation
    matrix = cv2.getPerspectiveTransform(input_points, converted_points)
    img_output = cv2.warpPerspective(img_original, matrix, (max_width, max_height))
    if img_output is None:
        raise Exception("Wrapping failed")
    

    #resized_image = cv2.resize(img_output, (600,400)) ## this is the one to be saved

    
    return img_output

In [6]:
#--------testing------
img_path = "C:\\Users\\USER\\Documents\\Grad_project\\Authentication\\Authentication_Grad_Project\\data\\Adel-Imam1.jpeg" 
img1_path = "C:\\Users\\USER\\Documents\\Grad_project\\Authentication\\Authentication_Grad_Project\\data\\rk.jpg"
img2_path = "C:\\Users\\USER\\Documents\\Grad_project\\Authentication\\Authentication_Grad_Project\\data\\wrap.output_resized.jpg"
img_p = get_img_from_path(img_path)
cv2.imshow('pic',img_p)

#contoured_pic = get_contours(img_p)
gray = cv2.cvtColor(img_p, cv2.COLOR_BGR2GRAY)

gray = cv2.bilateralFilter(gray, 20, 30, 30)
edged = cv2.Canny(gray, 10, 20)
contours, _ = cv2.findContours(edged.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
print(type(contours))
contours = sorted(contours, key=cv2.contourArea, reverse=True)[:10]
biggest = biggest_contour(contours)
cv2.imshow('edged',edged)
#contoured_pic = get_contours(edged)

#cv2.imshow('Result',contoured_pic)

while(1):
    k = cv2.waitKey(1)
    if k == ord('q'):
        break
cv2.destroyAllWindows()

<class 'tuple'>
16 approx
514.0 max_area
16 approx
14 approx
14 approx
13 approx
16 approx
18 approx
16 approx
10 approx
11 approx
4 len biggest
